In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import re
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
house_address_all = []
house_details_all = []
house_address = []
house_details = []
postal_code_array = []
latitude_array = []
longitude_array = []
year_built_array = []
payment_monthly = []
lot_size = []
basement = []
garage = []
mlsa = []
mls_array = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #time.sleep(5)
    
    for each in soup.find_all(class_= "card-text ng-star-inserted" ):
        try: 
            mls = each.find(class_='mlsNumber').text.replace('MLS® #: ', '')
            mls_array.append(mls)
        except:
            mls_array.append(np.nan)

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address_all.append(address.text)
    except:
        house_address_all.append('None')
        
    #time.sleep(5)
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details_all.append(detail.text)
    except:
        house_details_all.append('None')
        
    #time.sleep(5)
        
    house_details_more=[]
    url_house = []
        
    try:
        for detail in soup.find_all('a'):
            house_details_more.append(detail.get('href'))
    except:
        house_details_more.append('None')
    
    #time.sleep(5)

    for i in house_details_more:
        if '/ab/calgary-real-estate/' in str(i):
            link = 'https://www.remax.ca' + str(i)
            url_house.append(link)
            
    #time.sleep(5)
    
    for i in url_house:
    
        time.sleep(5)

        page = requests.get(i)
        soup = BeautifulSoup(page.text, "html.parser")

        #lat_all = soup.find_all('script')[3]
        lat3 = soup.find_all('script')[3]
        lat4 = soup.find_all('script')[4]
        
        string3 = ', '.join(lat3)
        string4 = ', '.join(lat4)
        
        #string = ', '.join(lat_all)
        #pc = "postalCode"
        #position_pc = string.find(pc)
        #pcd = string[(position_pc+14):(position_pc+20)]
        #postal_code_array.append(pcd)
        
        lat = "latitude"
        if lat in string3:
            position_lat = string3.find(lat)
            latd = string3[(position_lat+11):(position_lat+20)]
            latitude_array.append(latd)
        elif lat in string4:
            position_lat = string4.find(lat)
            latd = string4[(position_lat+11):(position_lat+20)]
            latitude_array.append(latd)

        long = "longitude"
        if long in string3:
            position_lat = string3.find(long)
            latd = string3[(position_lat+11):(position_lat+20)]
            longitude_array.append(latd)
        elif long in string4:
            position_lat = string4.find(long)
            latd = string4[(position_lat+11):(position_lat+20)]
            longitude_array.append(latd)

        house_address = []

        #time.sleep(5)

        try:
            addresses = soup.find_all('section', class_='property-details')
            for address in addresses:
                house_address.append(address.text)
        except:
            house_address.append('None')

        #time.sleep(5)

        string = ', '.join(house_address)
        s_year = "Year Built: "
        if s_year in string:            
            position_year = string.find(s_year)
            year_built = string[(position_year+12):(position_year+16)]
            year_built_array.append(year_built)
        else:
            year_built_array.append(np.nan)
        
        s_lot = "Lot Size: "
        if s_lot in string:
            try:
                start = string.rindex(s_lot) + len(s_lot)
                end = string.rindex("Occupancy", start )
                lot_size.append(string[start:end])
            except:
                lot_size.append(np.nan)
        else:
            lot_size.append(np.nan)
        
        basement_yn = "Basement"
        if basement_yn in string:
            basement.append(1)
        else:
            basement.append(0)
            
        garage_yn = "Garage"
        if garage_yn in string:
            garage.append(1)
        else:
            garage.append(0)
        
        try:
            payment = soup.find(class_="payment-amount").text
            payment_monthly.append(payment)
        except:
            payment_monthly.append(np.nan)
        
        #result = re.search('MLSÂ® #: (.*)Ownership', string)
        #result_app = result.group(1)
        #mlsa.append(result.group(1))
        
        mlsa_number = "MLSÂ® #: "
        try:
            start = string.rindex(mlsa_number) + len(mlsa_number)
            end = string.rindex("Ownership", start )
            mlsa.append(string[start:end])
        except:
            #mlsa_pos = string.find(mlsa_number)
            #mlsa_temp = string[(mlsa_pos+8):(mlsa_pos+16)]
            mlsa.append(np.nan)

        #time.sleep(5)

In [3]:
latitude_array[:5]

['51.143517', '51.060892', '51.041044', '51.103611', '51.00287,']

In [4]:
longitude_array[:5]

[' -114.152', ' -113.959', ' -114.029', ' -114.163', ' -114.129']

In [5]:
mls_array_df_temp = pd.DataFrame(mls_array)
mls_array_df_temp.head()

,0
0,MLSÂ® #: A1046648
1,MLSÂ® #: A1046959
2,MLSÂ® #: A1046689
3,MLSÂ® #: A1046941
4,MLSÂ® #: A1046898


In [6]:
mls_array_df = mls_array_df_temp[0].str.split("MLSÂ® #: ", 1, expand=True)
mls_array_df[1].head()

0    A1046648
1    A1046959
2    A1046689
3    A1046941
4    A1046898
Name: 1, dtype: object

In [7]:
address_df = pd.DataFrame(house_address_all)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")
new_df["price"] = pd.to_numeric(new_df["price"])

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"27 HAMPTONS HEATH NW, Calgary, AB, T3A 5E8",576800
1,"28 MARCOMBE PL NE, Calgary, AB, T2A 4H9",389900
2,"7 NEW ST SE, Calgary, AB, T2G 3X8",899900
3,"402 - 16 VARSITY ESTATES CIR NW, Calgary, AB, ...",455000
4,"1416 - 5605 HENWOOD ST SW, Calgary, AB, T3E 7R2",295000


In [8]:
final_df_pc = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df_pc.head()

,0,1
0,27 HAMPTONS HEATH NW,T3A 5E8
1,28 MARCOMBE PL NE,T2A 4H9
2,7 NEW ST SE,T2G 3X8
3,402 - 16 VARSITY ESTATES CIR NW,T3A 2C5
4,1416 - 5605 HENWOOD ST SW,T3E 7R2


In [9]:
#final_df_pc_clean = final_df_pc[1].str.replace(" ", "")
final_df_pc_clean = final_df_pc[1]
final_df_pc_clean.head()

0    T3A 5E8
1    T2A 4H9
2    T2G 3X8
3    T3A 2C5
4    T3E 7R2
Name: 1, dtype: object

In [10]:
df_add = pd.concat([new_df, final_df_pc_clean, final_df_pc[0], mls_array_df[1]], axis=1)
del df_add[2]
df_add.columns = ["price", "postal_code", "address", "mls"]
df_add.head()

,price,postal_code,address,mls
0,576800,T3A 5E8,27 HAMPTONS HEATH NW,A1046648
1,389900,T2A 4H9,28 MARCOMBE PL NE,A1046959
2,899900,T2G 3X8,7 NEW ST SE,A1046689
3,455000,T3A 2C5,402 - 16 VARSITY ESTATES CIR NW,A1046941
4,295000,T3E 7R2,1416 - 5605 HENWOOD ST SW,A1046898


In [11]:
details = pd.DataFrame(house_details_all)
details_df_temp = details[0].str.split('|', expand=True)
details_df_temp.head()

,0,1,2,3
0,4 bed,3 + 1 bath,1891 sqft,house
1,5 bed,2 + 1 bath,1133 sqft,house
2,4 bed,2 bath,1216 sqft,house
3,2 bed,2 bath,1281 sqft,condo
4,2 bed,2 bath,887 sqft,condo


In [12]:
details_df_bed = details_df_temp[0].str.replace(' bed', '')
details_df_bath = details_df_temp[1].str.replace(' bath', '')
details_df_area = details_df_temp[2].str.replace(' sqft', '')

In [13]:
details_df_bath_all = details_df_bath.str.split('+', expand=True)
details_df_bath_full = details_df_bath_all[0]
details_df_bath_half = details_df_bath_all[1]

In [14]:
details_df_bed = details_df_bed.replace('N/A', np.nan)
details_df_bed = pd.to_numeric(details_df_bed)
details_df_area = details_df_area.replace('N/A', np.nan)
details_df_area = pd.to_numeric(details_df_area)
details_df_bath_full = details_df_bath_full.replace('N/A', np.nan)
details_df_bath_full = pd.to_numeric(details_df_bath_full)
details_df_bath_half = details_df_bath_half.replace('N/A', int(0))
details_df_bath_half = pd.to_numeric(details_df_bath_half)

In [15]:
data = {'bed':details_df_bed, 'full_bath':details_df_bath_full, 'half_bath':details_df_bath_half,
       'property_area':details_df_area, 'property_type':details_df_temp[3]}

In [16]:
details_df = pd.DataFrame(data)
details_df.head()

,bed,full_bath,half_bath,property_area,property_type
0,4,3,1.0,1891,house
1,5,2,1.0,1133,house
2,4,2,NaN,1216,house
3,2,2,NaN,1281,condo
4,2,2,NaN,887,condo


In [17]:
calgary_df_dup = pd.concat([df_add, details_df], axis=1)
calgary_df = calgary_df_dup.drop_duplicates()
calgary_df.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type
0,576800,T3A 5E8,27 HAMPTONS HEATH NW,A1046648,4,3,1.0,1891,house
1,389900,T2A 4H9,28 MARCOMBE PL NE,A1046959,5,2,1.0,1133,house
2,899900,T2G 3X8,7 NEW ST SE,A1046689,4,2,NaN,1216,house
3,455000,T3A 2C5,402 - 16 VARSITY ESTATES CIR NW,A1046941,2,2,NaN,1281,condo
4,295000,T3E 7R2,1416 - 5605 HENWOOD ST SW,A1046898,2,2,NaN,887,condo


In [18]:
len(calgary_df)

20

In [19]:
calgary_df.to_csv('data/calgary_listings-a-test.csv', index=False)

## Cleanup Postal Codes, Latitude, Longitude and Year Built

postal_codes_messy = pd.DataFrame(postal_code_array, columns=["postal_code"])

postal_codes_messy.head()

postal_code_array_clean = postal_codes_messy["postal_code"].str.replace('"', '')
postal_code_array_clean = postal_code_array_clean.str.replace(" ", "")

In [20]:
latitude_array_messy = pd.DataFrame(latitude_array, columns=["latitude"])

In [21]:
#latitude_array_messy

In [22]:
latitude_array_messy = latitude_array_messy["latitude"].str.replace(',', '')
latitude_array_messy.head()

0    51.143517
1    51.060892
2    51.041044
3    51.103611
4     51.00287
Name: latitude, dtype: object

In [23]:
longitude_array_messy = pd.DataFrame(longitude_array, columns=["longitude"])

In [24]:
longitude_array_messy = longitude_array_messy["longitude"].str.replace(',', '')
longitude_array_messy.head()

0     -114.152
1     -113.959
2     -114.029
3     -114.163
4     -114.129
Name: longitude, dtype: object

<div class="alert alert-danger">
    <b>Unused</b>
    
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])<br>
year_built_clean = year_built_messy["year_built"].str.replace('', np.nan)<br>
year_built_num = year_built_messy.astype(int)

year_built_num.head()
</div>

In [25]:
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])
#year_built_messy = pd.to_numeric(year_built_messy)
year_built_messy.head()

,year_built
0,1993
1,1973
2,1914
3,2015
4,2006


In [26]:
year_built_clean = year_built_messy["year_built"].replace('', np.nan)

In [27]:
year_built_clean.head(11)

0     1993
1     1973
2     1914
3     2015
4     2006
5     1964
6     1961
7     1964
8     2017
9     1976
10    2011
Name: year_built, dtype: object

In [28]:
year_built_digits = []

for i in year_built_clean:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        #year_built_clean.replace(i, np.nan)
        year_built_digits.append(np.nan)
    if ((str(i) == ' 9 2') or (str(i) == ' 8 2') or (str(i) == ' 2 2') or (str(i) == 'nasq') or (str(i) == ' 6 R') or (str(i) == ' 6 S')):
        year_built_digits.append(np.nan)
    else:
        year_built_digits.append(i)

In [29]:
year_built_digits[:11]

['1993',
 '1973',
 '1914',
 '2015',
 '2006',
 '1964',
 '1961',
 '1964',
 '2017',
 '1976',
 '2011']

In [30]:
year_built_num = pd.to_numeric(year_built_digits)
year_built_num[:10]

array([1993., 1973., 1914., 2015., 2006., 1964., 1961., 1964., 2017.,
       1976.])

In [31]:
year_built_num_pd = pd.DataFrame(year_built_num)

In [32]:
years_old = 2020 - year_built_num
years_old[:10]

array([ 27.,  47., 106.,   5.,  14.,  56.,  59.,  56.,   3.,  44.])

In [33]:
years_old_df = pd.DataFrame(years_old)

In [34]:
latitude_digits = []

for i in latitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        latitude_digits.append(np.nan)
    else:
        latitude_digits.append(i)

In [35]:
longitude_digits = []

for i in longitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        longitude_digits.append(np.nan)
    else:
        longitude_digits.append(i)

In [36]:
latitude_digits = pd.to_numeric(latitude_digits)
longitude_digits = pd.to_numeric(longitude_digits)

In [37]:
payment_monthly_messy = pd.DataFrame(payment_monthly, columns=["monthly_payments"])
payment_monthly_messy.head()

,monthly_payments
0,$2494.15
1,$1685.98
2,$3891.28
3,$1967.48
4,$1275.62


In [38]:
payment_monthly_messy = payment_monthly_messy["monthly_payments"].str.replace('$', '')
payment_monthly_messy.head(10)

0    2494.15
1    1685.98
2    3891.28
3    1967.48
4    1275.62
5    2269.73
6    2118.39
7    2443.13
8    1988.66
9    1751.27
Name: monthly_payments, dtype: object

In [39]:
payment_monthly_messy = payment_monthly_messy.replace('', np.nan)

In [40]:
payment_monthly_clean = payment_monthly_messy.replace('None', np.nan)
payment_monthly_clean = pd.to_numeric(payment_monthly_clean)

<div class="alert alert-success">
    <b>Guide</b>
    
https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

#### approximate radius of earth in km
R = 6373.0
    
lat1 = radians(52.2296756)<br>
lon1 = radians(21.0122287)

lat2 = radians(52.406374)<br>
lon2 = radians(16.9251681)

dlon = lon2 - lon1<br>
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2<br>
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
    
#### Calgary Downtown coordinates (51.0449485,-114.0739559)
</div>

In [41]:
distance_downtown = []

for i, j in zip(latitude_digits, longitude_digits):
    lat1 = radians(i)
    lon1 = radians(j)
    lat2 = radians(51.0449485)
    lon2 = radians(-114.0739559)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6373.0
    distance = (R * c)
    
    distance_downtown.append(round(distance, 2))

In [42]:
distance_downtown[:5]

[12.24, 8.23, 3.17, 9.02, 6.06]

In [43]:
distance_downtown_df = pd.DataFrame(distance_downtown)

<div class="alert alert-warning">
    <b>Another example</b>
    
import geopy.distance

coords_1 = (52.2296756, 21.0122287)<br>
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)
</div>

In [44]:
more_details = pd.DataFrame([latitude_digits, longitude_digits, year_built_num]).T
#more_details.columns = ["postal_code", "latitude", "longitude", "year_built"]
more_details.head()

,0,1,2
0,51.143517,-114.152,1993.0
1,51.060892,-113.959,1973.0
2,51.041044,-114.029,1914.0
3,51.103611,-114.163,2015.0
4,51.002870,-114.129,2006.0


In [45]:
lot_size_df = pd.DataFrame(lot_size)

In [46]:
basement_df = pd.DataFrame(basement)

In [47]:
garage_df = pd.DataFrame(garage)

In [48]:
mls_df = pd.DataFrame(mlsa)

In [49]:
more_df = pd.concat([more_details, years_old_df, distance_downtown_df, payment_monthly_clean, lot_size_df, basement_df, garage_df, mls_df], axis=1)
more_df.columns = ["latitude", "longitude", "year_built", "years_old", "distance_downtown", "monthly_payments", "lot_size", "basement", "garage", "mls"]
more_df.head()

,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage,mls
0,51.143517,-114.152,1993.0,27.0,12.24,2494.15,8654,1.0,1.0,A1046648
1,51.060892,-113.959,1973.0,47.0,8.23,1685.98,11313,1.0,1.0,A1046959
2,51.041044,-114.029,1914.0,106.0,3.17,3891.28,5253,1.0,0.0,A1046689
3,51.103611,-114.163,2015.0,5.0,9.02,1967.48,NaN,0.0,1.0,A1046941
4,51.002870,-114.129,2006.0,14.0,6.06,1275.62,NaN,0.0,1.0,A1046898


In [50]:
len(more_df)

63

In [51]:
more_df.to_csv('data/calgary_listings-b-test.csv', index=False)

In [52]:
result_dup = pd.merge(calgary_df, more_df, on='mls')
result = result_dup.drop_duplicates()
result = result.reset_index(drop=True)
result.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,576800,T3A 5E8,27 HAMPTONS HEATH NW,A1046648,4,3,1.0,1891,house,51.143517,-114.152,1993.0,27.0,12.24,2494.15,8654,1.0,1.0
1,576800,T3A 5E8,27 HAMPTONS HEATH NW,A1046648,4,3,1.0,1891,house,51.143517,-114.152,2008.0,12.0,12.24,2494.15,8654,1.0,1.0
2,576800,T3A 5E8,27 HAMPTONS HEATH NW,A1046648,4,3,1.0,1891,house,51.143517,-114.152,1975.0,45.0,12.24,2494.15,8654,1.0,1.0
3,389900,T2A 4H9,28 MARCOMBE PL NE,A1046959,5,2,1.0,1133,house,51.060892,-113.959,1973.0,47.0,8.23,1685.98,11313,1.0,1.0
4,389900,T2A 4H9,28 MARCOMBE PL NE,A1046959,5,2,1.0,1133,house,51.060892,-113.959,1993.0,27.0,8.23,1685.98,11313,1.0,1.0


In [53]:
len(result)

54

In [54]:
result_df_clean = result.sort_values('address').drop_duplicates(subset=['mls']) 
result_df_clean.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
39,225000,T3H 4C4,"102, 2188 85 Street SW",A1043290,2,1,NaN,634,townhouse,51.035997,-114.210,2002.0,18.0,9.57,972.93,na,0.0,0.0
14,295000,T3E 7R2,1416 - 5605 HENWOOD ST SW,A1046898,2,2,NaN,887,condo,51.002870,-114.129,1914.0,106.0,6.06,1275.62,NaN,0.0,1.0
47,249900,T3G 5T2,2204 - 928 ARBOUR LAKE RD NW,A1046820,2,2,NaN,1061,condo,51.132670,-114.215,2022.0,-2.0,13.87,1080.60,NaN,0.0,1.0
53,279900,T3H 0K1,2209 - 211 ASPEN STONE BLVD SW,A1046610,2,2,1.0,1207,condo,51.038162,-114.215,2020.0,0.0,9.89,1210.32,NaN,0.0,0.0
28,405000,T2W 2W7,223 BROOKPARK DR SW,A1046755,5,1,1.0,1150,house,50.955108,-114.113,1964.0,56.0,10.36,1751.27,531.00,1.0,0.0


In [55]:
len(result_df_clean)

19

In [56]:
result_df_clean.to_csv('data/calgary_listings-test.csv', index=False)

-----------

<div class="alert alert-info">
    <b>UNSED</b>
</div>

In [ ]:
house_details_more=[]
url_house = []
try:
    for detail in soup.find_all('a'):
        house_details_more.append(detail.get('href'))
except:
    house_details_more.append('None')
    
for i in house_details_more:
    if '/ab/calgary-real-estate/' in str(i):
        link = 'https://www.remax.ca' + str(i)
        url_house.append(link)
url_house

In [ ]:
url_house[:5]

In [ ]:
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

for i in url_house[:5]:
    
    time.sleep(5)
    
    page = requests.get(i)
    soup = BeautifulSoup(page.text, "html.parser")
    
    lat_all = soup.find_all('script')[4]
    
    string = ', '.join(lat_all)
    pc = "postalCode"
    position_pc = string.find(pc)
    pcd = string[(position_pc+14):(position_pc+21)]
    PostalCode.append(pcd)

    lat = "latitude"
    position_lat = string.find(lat)
    latd = string[(position_lat+11):(position_lat+20)]
    Latitude.append(latd)

    long = "longitude"
    position_long = string.find(long)
    longd = string[(position_long+12):(position_long+23)]
    Longitude.append(longd)
    
    house_address = []
    
    time.sleep(5)
    
    try:
        addresses = soup.find_all('section', class_='property-details')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
    
    time.sleep(5)
    
    string = ', '.join(house_address)
    s_year = "Year Built"
    position_year = string.find(s_year)
    year_built = string[(position_year+12):(position_year+16)]
    YearBuilt.append(year_built)